In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy as np
import pmdarima as pm

from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

In [ ]:
df = pd.read_parquet("../data/tank1_new/oxygen_a.parquet")
df.index = pd.to_datetime(df['datumBeginMeting'])
df.index.name = None
df.drop(columns=['datumEindeMeting', 'datumBeginMeting'], inplace=True)

df = df["hstWaarde"]

df

In [ ]:
# Plot the time series data
plt.figure(figsize=(12, 6))
plt.plot(df)
plt.title('Values')
plt.xlabel('Time')
plt.ylabel('Value')
plt.show()

In [ ]:
# Check for stationarity using Augmented Dickey-Fuller test
# If p-value is less than 0.05, data is stationary, otherwise, it's not.
result = adfuller(df['hstWaarde'])
print("ADF Statistic:", result[0])
print("p-value:", result[1])
print("Critical Values:", result[4])

In [ ]:
df = df.astype(float)
df

df2 = df.iloc[len(df) // 2:]
df2

In [ ]:
# Perform a train-test split
train_size = int(len(df2) * 0.8)  # 80% for training, adjust as needed
train, test = df2.iloc[:train_size], df2.iloc[train_size:]

# Fit your model
model = pm.auto_arima(train, seasonal=True, m=1)

# make your forecasts
forecasts = model.predict(test.shape[0])  # predict N steps into the future

# Visualize the forecasts (blue=train, green=forecasts)
x = np.arange(df2.shape[0])
plt.plot(x[:train_size], train, c='blue')
plt.plot(x[train_size:], forecasts, c='green')
plt.show()

In [ ]:
# df = df.resample("5T").mean()

# Perform a train-test split
train_size = int(len(df) * 0.8)  # 80% for training, adjust as needed
train_data, test_data = df.iloc[:train_size], df.iloc[train_size:]

sarima_train = SARIMAX(train_data,
                order=(1,1,1),
                seasonal_order=(1,1,1,4))

sarima_fit = sarima_train.fit()

test_predictions = sarima_fit.get_prediction(start=len(train_data), end=len(df) - 1, dynamic=False)

# Extract the predicted values and confidence intervals
predicted_values = test_predictions.predicted_mean
confidence_intervals = test_predictions.conf_int()

plt.figure(figsize=(16, 4))
plt.plot(test_data, label="Actual")
plt.plot(predicted_values, label="Predicted")
plt.fill_between(confidence_intervals.index, confidence_intervals.iloc[:, 0], confidence_intervals.iloc[:, 1], color='gray', alpha=0.2)
plt.title('', fontsize=20)
plt.ylabel('', fontsize=16)
plt.legend()

In [ ]:
predicted_values

In [ ]:
# Perform a train-test split
train_size = int(len(df) * 0.8)  # 80% for training, adjust as needed
train_data, test_data = df.iloc[:train_size], df.iloc[train_size:]

# Fit the ARIMA model with (2, 0, 2) parameters
# p: the number of lag observations in the model, also known as the lag order. 
# d: the number of times the raw observations are differenced
# q: the size of the moving average window, also known as the order of the moving average
model = sm.tsa.ARIMA(train_data, order=(1, 1, 1))
model_fit = model.fit()

# Make predictions on the test set
predictions = model_fit.forecast(steps=len(test_data))

# Calculate Mean Squared Error (MSE) to evaluate the model
mse = mean_squared_error(test_data, predictions)
print(f'Mean Squared Error (MSE): {mse}')

# Plot the original data and predictions
plt.figure(figsize=(12, 6))
plt.plot(df, label='Original Data', color='blue')
plt.plot(test_data.index, predictions, label='Predictions', color='red')
plt.legend()
plt.xlabel('Timestamp')
plt.ylabel('Value')
plt.title('ARIMA Model with (2, 0, 2) Parameters')
plt.show()

In [ ]:
predictions

In [ ]:
# Fit an ARIMA model
p = 1  # AR order
d = 1  # I order (differencing)
q = 1  # MA order

model = sm.tsa.ARIMA(df['hstWaarde'], order=(2,0,2))
results = model.fit()

# Print the model summary
print(results.summary())

In [ ]:
# Plot the model residuals
plt.figure(figsize=(12, 6))
plt.plot(results.resid)
plt.title('Residuals of ARIMA Model')
plt.xlabel('Time')
plt.ylabel('Residuals')
plt.show()

In [ ]:
# Generate forecasts
res = results.forecast(10)
forecast = res.keys()
stderr = res

# Plot the original data and forecasted values
plt.figure(figsize=(12, 6))
plt.plot(df['hstWaarde'], label='Original Data')
plt.plot(res, label="as")
# plt.plot(range(len(df), len(df) + 10), forecast, label='Forecast', color='red')
# plt.fill_between(range(len(df), len(df) + 10), forecast - stderr, forecast + stderr, color='pink')
plt.legend()
plt.title('ARIMA Forecast')
plt.xlabel('Time')
plt.ylabel('Value')
plt.show()